加载config example

In [1]:
import json
from loguru import logger
import pandas as pd

'''
训练和推理的datasource不同，因此
测试训练的config需要在config中修改mode = "TRAIN"， 并在下方选择config-train-example.json
测试推理的config需要在config中修改mode = "INFERENCE"，并在下方选择config-inference-example.json
'''
# config_file_path = "../config-train-example.json"
config_file_path = "../config-inference-example.json"
logger.info("config file is : {}".format(config_file_path))

with open(config_file_path) as config_file:
    config_content = config_file.read()

json_config = json.loads(config_content)
print(json_config)

2023-02-24 21:22:28.685 | INFO     | __main__:<cell line: 7>:7 - config file is : ../config-inference-example.json


{'hyperParams': {'batch_size': 32, 'epochs': 10}, 'modelPath': 'testModelPath', 'resultPath': 'result', 'trainJobName': 'nginx-pure-data-ecl-64778820', 'datasetStartTime': -1, 'ckptName': 'nginx-pure_data.ECL-64778820', 'storage': [{'type': 'REDIS', 'usage': 'model', 'host': 'localhost', 'port': 6379, 'password': '', 'properties': {'db': 0}}, {'type': 'MONGODB', 'usage': 'result', 'host': '10.112.169.94', 'port': 30332, 'username': 'root', 'password': 'root', 'properties': {'db': 'evaluation'}}], 'selectedFields': ['MT_000', 'MT_001', 'date'], 'algorithm': {'type': 'KNN', 'name': 'nginx', 'description': 'nginx', 'author': 'wanz', 'version': '1.0', 'url': 'nginx', 'hyperParameters': {'hello': 'world'}, 'command': ' ', 'hashString': '104760218104760218364187848563104760218197974952332', 'id': '63f320a11a1cf03bbd8f01c6'}, 'datasetEndTime': -1, 'dataSource': {'type': 'KAFKA', 'host': '192.168.31.197', 'port': 9092, 'username': 'root', 'password': 'root', 'properties': {'topic': 'node_metri

测试config基本字段

In [2]:
from sdk.config.config import Config

config = Config(json_config)
print("selected fields: {}".format(config.selected_fields))
print("algorithm name: {}".format(config.algorithm.get_name()))

2023-02-24 21:22:28.768 | INFO     | sdk.config.config:__init__:21 - current mode is INFERENCE


selected fields: ['MT_000', 'MT_001', 'date']
algorithm name: nginx


测试storage

In [3]:
# model storage and redis storage
model_storage = config.get_model_storage()


class TestModel:
    def __init__(self):
        self.data = 1

    def get_data(self):
        return self.data


model = TestModel()

model_storage.save_model(config.model_path, model, {"min": 1, "max": 2})
model_retrieve, args = model_storage.load_model(config.model_path)
print(model_retrieve.get_data())
print(args)

# result storage and mongo storage
result_storage = config.get_result_storage()
result_storage.store_result(config.result_path, "test-algo", ["test-metric1"], ["test-target1"], pd.Series(["1"]))

1
{'min': 1, 'max': 2}


测试datasource

In [4]:
# train datasource and mongo datasource
if "train" in config_file_path:
    train_datasource = config.get_train_datasource()
    train_datasource.set_default_projected_fields({"_id": 0})
    train_datasource.set_default_df_index("date")
    data = train_datasource.get_data(config.dataset_start_time, config.dataset_end_time, config.selected_fields)

# inference datasource and kafka datasource
elif "inference" in config_file_path:
    inference_datasource = config.get_inference_datasource()
    stream = inference_datasource.get_stream_handler()

    count = 0
    for message in stream:
        mvalue = message.value
        print(mvalue)
        count += 1
        if count == 5:
            break

{'cmdb_id': 'k8s-large-1665988619', 'name': 'node_filefd_allocated', 'timestamp': '2023-02-24T13:22:37Z', 'value': '8288'}
{'cmdb_id': 'k8s-large-1665988619', 'name': 'node_load1', 'timestamp': '2023-02-24T13:22:37Z', 'value': '0.36'}
{'cmdb_id': 'k8s-large-1665988619', 'name': 'node_load15', 'timestamp': '2023-02-24T13:22:37Z', 'value': '0.37'}
{'cmdb_id': 'k8s-large-1665988619', 'name': 'node_load5', 'timestamp': '2023-02-24T13:22:37Z', 'value': '0.43'}
{'cmdb_id': 'k8s-large-1665988619', 'name': 'node_memory_Active_anon_bytes', 'timestamp': '2023-02-24T13:22:37Z', 'value': '6368419840'}
